# Setup Environnement

In [1]:
import numpy as np
from numpy import ceil
import pandas as pd

import json
from pprint import pprint

import unicodedata
import spacy
from collections import defaultdict

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from gensim import corpora, models

import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.gensim



c:\Users\prince.mezuirotimi\OneDrive - Gedeon - SIPAOF\Documents\PERSONNEL\ENSAI\MESURE ET ANALYSE DE LA SATISFACTION CLIENT\venv-Satisfaction\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Télécharger le modèle de langue français de spaCy
# !python -m spacy download fr_core_news_sm

# Charger le modèle de langue français de spaCy
nlp = spacy.load("fr_core_news_sm")


# Données

## Importation

In [ ]:
# Lecture du fichier JSON
with open("tourisme_responsable.json", 'r', encoding='utf-8') as file:
    base = json.load(file)

# Création de la base d'étude
base = base['results']

## Exploration & Pré-traitement

**CONSIGNES** :

> Office du tourisme d’Ille-et-Vilaine : **“Quelles actions simples pourrait-on mener en priorité pour tendre vers un tourisme responsable ?”**

Consultation citoyenne “*comment agir pour un tourisme plus durable en France*” (1500 répondants)



Le jeu de données à notre disposition présente les différentes propositions faites dans le cadre du sujet énoncé au-dessus.

Pour chaque proposition, on a les informations sur l'auteur, et le résumé des votes des sondés.

A chaque vote, un sondé doit choisir entre un vote pour, neutre et négatif. Et pour chaque vote fait il a le choix de preciser son vote par un sous-vote donnant de l'intensité à son vote associé.

In [4]:
# Aperçu de la base
pprint(base[0], sort_dicts=False)

{'id': '67d0c562-fb00-4228-aa08-e504a8077c38',
 'userId': '93ef60de-5827-49f1-bd56-e1f580619540',
 'content': 'Il faut favoriser le stationnement des véhicules de loisir proche '
            "des sites touristiques au même titre qu'un véhicule léger.",
 'contentLanguage': 'fr',
 'translatedContent': None,
 'translatedLanguage': None,
 'slug': 'il-faut-favoriser-le-stationnement-des-vehicules-de-loisir-proche-des-sites-touristiques-au-meme-titre-qu-un-vehicule-leger',
 'status': 'Accepted',
 'createdAt': '2021-06-04T08:36:40.430Z',
 'updatedAt': '2021-08-19T12:29:37.892Z',
 'votes': [{'voteKey': 'agree',
            'count': 46,
            'score': 0.37,
            'qualifications': [{'qualificationKey': 'likeIt',
                                'count': 8,
                                'hasQualified': False},
                               {'qualificationKey': 'platitudeAgree',
                                'count': 2,
                                'hasQualified': False},
     

In [5]:
# Création du DataFrame des données
data = []

for i in range(len(base)):
    # On extrait les données de la ligne i
    id = base[i]['id']
    
    user_id = base[i]['userId']
    user_name = base[i]['author']['firstName']
    user_age = base[i]['author']['age']
    user_type = base[i]['author']['userType']
    user_cp = base[i]['author']['postalCode']
    
    content = base[i]['content']
    slug = base[i]['slug']
    
    count_agree = base[i]['votes'][0]['count']
    count_like_it = base[i]['votes'][0]['qualifications'][0]['count']
    count_platitude_agree = base[i]['votes'][0]['qualifications'][1]['count']
    count_doable = base[i]['votes'][0]['qualifications'][2]['count']

    count_neutral = base[i]['votes'][1]['count']
    count_no_opinion = base[i]['votes'][1]['qualifications'][0]['count']
    count_dn_understand = base[i]['votes'][1]['qualifications'][0]['count']
    count_dn_care = base[i]['votes'][1]['qualifications'][0]['count']
    
    count_disagree = base[i]['votes'][2]['count']
    count_impossible = base[i]['votes'][2]['qualifications'][0]['count']
    count_no_way = base[i]['votes'][2]['qualifications'][1]['count']
    count_platitude_disagree = base[i]['votes'][2]['qualifications'][2]['count']
    

    
    # On ajoute la ligne i au DataFrame
    data.append({'ID': id, 'USER_ID': user_id, 'USER_NAME': user_name, 'USER_AGE': user_age,
                      'USER_TYPE': user_type, 'USER_CP': user_cp,
                      'CONTENT': content, 'SLUG': slug,
                      'COUNT_AGREE': count_agree, 'COUNT_LIKE_IT': count_like_it, 'COUNT_PLATITUDE_AGREE': count_platitude_agree, 'COUNT_DOABLE': count_doable,
                      'COUNT_NEUTRAL': count_neutral, 'COUNT_NO_OPINION': count_no_opinion, 'COUNT_DN_UNDERSTAND': count_dn_understand, 'COUNT_DN_CARE': count_dn_care, 
                      'COUNT_DISAGREE': count_disagree, 'COUNT_IMPOSSIBLE': count_impossible, 'COUNT_NO_WAY': count_no_way, 'COUNT_PLATITUDE_DISAGREE': count_platitude_disagree
                      })

data = pd.DataFrame(data)



In [6]:
# Ajout de quelques informations supplémentaires
# Création des colonnes USER_DEPARTEMENT et USER_REGION
data['USER_DEPARTEMENT'] = data['USER_CP'].apply(lambda x: str(x)[:2])

dict_dep_region = {
    '01': 'Auvergne-Rhône-Alpes',
    '02': 'Hauts-de-France',
    '03': 'Auvergne-Rhône-Alpes',
    '04': 'Provence-Alpes-Côte d\'Azur',
    '05': 'Provence-Alpes-Côte d\'Azur',
    '06': 'Provence-Alpes-Côte d\'Azur',
    '07': 'Auvergne-Rhône-Alpes',
    '08': 'Grand Est',
    '09': 'Occitanie',
    '10': 'Grand Est',
    '11': 'Occitanie',
    '12': 'Occitanie',
    '13': 'Provence-Alpes-Côte d\'Azur',
    '14': 'Normandie',
    '15': 'Auvergne-Rhône-Alpes',
    '16': 'Nouvelle-Aquitaine',
    '17': 'Nouvelle-Aquitaine',
    '18': 'Centre-Val de Loire',
    '19': 'Nouvelle-Aquitaine',
    '2A': 'Corse',
    '2B': 'Corse',
    '21': 'Bourgogne-Franche-Comté',
    '22': 'Bretagne',
    '23': 'Nouvelle-Aquitaine',
    '24': 'Nouvelle-Aquitaine',
    '25': 'Bourgogne-Franche-Comté',
    '26': 'Auvergne-Rhône-Alpes',
    '27': 'Normandie',
    '28': 'Centre-Val de Loire',
    '29': 'Bretagne',
    '30': 'Occitanie',
    '31': 'Occitanie',
    '32': 'Occitanie',
    '33': 'Nouvelle-Aquitaine',
    '34': 'Occitanie',
    '35': 'Bretagne',
    '36': 'Centre-Val de Loire',
    '37': 'Centre-Val de Loire',
    '38': 'Auvergne-Rhône-Alpes',
    '39': 'Bourgogne-Franche-Comté',
    '40': 'Nouvelle-Aquitaine',
    '41': 'Centre-Val de Loire',
    '42': 'Auvergne-Rhône-Alpes',
    '43': 'Auvergne-Rhône-Alpes',
    '44': 'Pays de la Loire',
    '45': 'Centre-Val de Loire',
    '46': 'Occitanie',
    '47': 'Nouvelle-Aquitaine',
    '48': 'Occitanie',
    '49': 'Pays de la Loire',
    '50': 'Normandie',
    '51': 'Grand Est',
    '52': 'Grand Est',
    '53': 'Pays de la Loire',
    '54': 'Grand Est',
    '55': 'Grand Est',
    '56': 'Bretagne',
    '57': 'Grand Est',
    '58': 'Bourgogne-Franche-Comté',
    '59': 'Hauts-de-France',
    '60': 'Hauts-de-France',
    '61': 'Normandie',
    '62': 'Hauts-de-France',
    '63': 'Auvergne-Rhône-Alpes',
    '64': 'Nouvelle-Aquitaine',
    '65': 'Occitanie',
    '66': 'Occitanie',
    '67': 'Grand Est',
    '68': 'Grand Est',
    '69': 'Auvergne-Rhône-Alpes',
    '70': 'Bourgogne-Franche-Comté',
    '71': 'Bourgogne-Franche-Comté',
    '72': 'Pays de la Loire',
    '73': 'Auvergne-Rhône-Alpes',
    '74': 'Auvergne-Rhône-Alpes',
    '75': 'Île-de-France',
    '76': 'Normandie',
    '77': 'Île-de-France',
    '78': 'Île-de-France',
    '79': 'Nouvelle-Aquitaine',
    '80': 'Hauts-de-France',
    '81': 'Occitanie',
    '82': 'Occitanie',
    '83': 'Provence-Alpes-Côte d\'Azur',
    '84': 'Provence-Alpes-Côte d\'Azur',
    '85': 'Pays de la Loire',
    '86': 'Nouvelle-Aquitaine',
    '87': 'Nouvelle-Aquitaine',
    '88': 'Grand Est',
    '89': 'Bourgogne-Franche-Comté',
    '90': 'Bourgogne-Franche-Comté',
    '91': 'Île-de-France',
    '92': 'Île-de-France',
    '93': 'Île-de-France',
    '94': 'Île-de-France',
    '95': 'Île-de-France',
    '97': 'Outre-Mer',
    '98': 'Autre',
    '99' : 'Etranger',
}

data['USER_REGION'] = data['USER_DEPARTEMENT'].map(dict_dep_region)
data['USER_REGION'] = data['USER_REGION'].fillna('Autre')

In [7]:
# Modification du champ CONTENT
# Dans le champ CONTENT on retire la formule 'il faut ' qu'il y a au debut 
# data['CONTENT'] = data['CONTENT'].apply(lambda x: str(x)[8:])


In [8]:
data

,ID,USER_ID,USER_NAME,USER_AGE,USER_TYPE,USER_CP,CONTENT,SLUG,COUNT_AGREE,COUNT_LIKE_IT,...,COUNT_NEUTRAL,COUNT_NO_OPINION,COUNT_DN_UNDERSTAND,COUNT_DN_CARE,COUNT_DISAGREE,COUNT_IMPOSSIBLE,COUNT_NO_WAY,COUNT_PLATITUDE_DISAGREE,USER_DEPARTEMENT,USER_REGION
0,67d0c562-fb00-4228-aa08-e504a8077c38,93ef60de-5827-49f1-bd56-e1f580619540,Dominique,64.0,USER,None,Il faut favoriser le stationnement des véhicul...,il-faut-favoriser-le-stationnement-des-vehicul...,46,8,...,38,6,6,6,30,4,13,4,No,Autre
1,9798263d-96a3-4e35-9ba2-f15c6f6402d8,dd920812-f535-438d-a1d2-f7c72a24bc98,bosco,20.0,USER,None,Il faut éviter la course aux équipements et à ...,il-faut-eviter-la-course-aux-equipements-et-a-...,52,9,...,28,7,7,7,33,4,11,5,No,Autre
2,885d953e-a29a-4323-96d8-f149d3e512c7,38cf0349-3b79-4f4a-8c8f-14bfe1d3a854,Virginie,49.0,USER,63000,Il faut remettre en service les petites lignes...,il-faut-remettre-en-service-les-petites-lignes...,591,166,...,96,37,37,37,90,43,23,13,63,Auvergne-Rhône-Alpes
3,6a660958-f5c7-44ce-83de-d165263c8986,c7846aa6-6c28-4d27-8f75-4eb3f21c2054,Sabrina,NaN,USER,None,Il faut taxer de manière significative tous le...,il-faut-taxer-de-maniere-significative-tous-le...,91,26,...,24,5,5,5,36,7,19,7,No,Autre
4,fb259cf7-ec19-459b-a6ad-2254689b14c3,39ae8c6b-e853-48a9-ad03-150ff2009f9e,Bruno,61.0,USER,31240,Il faut un questionnaire obligatoire à chaque ...,il-faut-un-questionnaire-obligatoire-a-chaque-...,70,11,...,53,15,15,15,77,29,20,11,31,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,01fc2034-32a7-4bf2-8819-41d4da2b7781,76a6dc53-8cde-4bf1-8f3b-aadfa9eda086,Clémence,36.0,USER,None,"Il faut ""éduquer"" les voyageurs sur les choix ...",il-faut-eduquer-les-voyageurs-sur-les-choix-ex...,35,7,...,7,2,2,2,11,3,5,4,No,Autre
1492,c1331531-03e6-4d16-8466-f9b23196c842,1b66a5a4-34b8-4e4d-b87a-085fcb89aee1,Yann,50.0,USER,75008,Il faut réduire l'utilisation de la voiture in...,il-faut-reduire-l-utilisation-de-la-voiture-in...,67,14,...,18,2,2,2,42,19,13,8,75,Île-de-France
1493,9eb50171-5678-44db-ad69-2ffab2121553,b2b6e313-a1b3-4ed8-810c-130f0652b8db,Marie,30.0,USER,None,Il faut interdire la baignade sur les plages q...,il-faut-interdire-la-baignade-sur-les-plages-q...,56,11,...,28,6,6,6,21,8,9,11,No,Autre
1494,5b423680-f77b-42aa-adfe-c32e6bc1c1d3,92858108-3377-410c-9c2d-2c3318d3e24c,Nicolas,NaN,USER,None,Il faut que les intercommunalités aient un pla...,il-faut-que-les-intercommunalites-aient-un-pla...,39,6,...,14,1,1,1,10,7,3,4,No,Autre


## Pré-traitement des contenus

In [ ]:
# Ajouter de mots aux stop words de spaCy
custom_stopwords = [
    "faut", # pour retirer la formule 'il faut' qu'il y a au debut de chaque prooposition 
    "touriste", "touristes", "tourisme", "touristique", "touristiques"
    ] # Liste personnalisée des mots à exclure



for word in custom_stopwords:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True
    nlp.Defaults.stop_words.add(word)


In [10]:
# Fonction pour uniformiser, tokeniser et lemmatiser
def preprocess_text(text):
    # Fonction pour retirer les accents
    def retirer_accents(texte):
        texte_sans_accents = unicodedata.normalize('NFD', texte)
        texte_sans_accents = ''.join(
            c for c in texte_sans_accents if unicodedata.category(c) != 'Mn'
        )
        return texte_sans_accents

    text = retirer_accents(text)  # Retirer les accents
    doc = nlp(text.lower())  # Uniformisation (minuscule) et analyse
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]  # Lemmatisation et suppression des stopwords/punctuations
    return " ".join(tokens)

# Appliquer la fonction à la colonne CONTENT
data['CONTENT_PROCESSED'] = data['CONTENT'].apply(preprocess_text)

In [11]:
data[['CONTENT', 'CONTENT_PROCESSED']]

,CONTENT,CONTENT_PROCESSED
0,Il faut favoriser le stationnement des véhicul...,favoriser stationnement vehicule loisir site t...
1,Il faut éviter la course aux équipements et à ...,eviter course equipement densification offre h...
2,Il faut remettre en service les petites lignes...,remettre service petit ligne sncf interieure f...
3,Il faut taxer de manière significative tous le...,taxer maniere significatif vol court national ...
4,Il faut un questionnaire obligatoire à chaque ...,questionnaire obligatoire reservation informe ...
...,...,...
1491,"Il faut ""éduquer"" les voyageurs sur les choix ...",eduquer voyageur choix existant limiter emprei...
1492,Il faut réduire l'utilisation de la voiture in...,reduire utilisation voiture individuel encoura...
1493,Il faut interdire la baignade sur les plages q...,interdire baignade plage eau trop pollue usage...
1494,Il faut que les intercommunalités aient un pla...,intercommunalite avoir plan interconnexion dec...


# Extraction des thèmes

## Analyse des thèmes

In [12]:
# Dictionnaire et corpus BoW
text_data = [doc.split() for doc in data['CONTENT_PROCESSED']]
dictionary = corpora.Dictionary(text_data)
corpus_bow = [dictionary.doc2bow(text) for text in text_data]


In [13]:
from gensim.models import CoherenceModel

# Définir les plages de valeurs à tester
num_topics_range = [5, 10, 15, 20]
alpha_values = ['symmetric', 'asymmetric']
eta_values = ['symmetric']
passes_range = [10, 20]
iterations_range = [50]

# Stocker les résultats
results = []

# Boucle sur toutes les combinaisons
for num_topics in num_topics_range:
    for alpha in alpha_values:
        for eta in eta_values:
            for passes in passes_range:
                for iterations in iterations_range:
                    lda_model = models.LdaModel(
                        corpus=corpus_bow,
                        id2word=dictionary,
                        num_topics=num_topics,
                        alpha=alpha,
                        eta=eta,
                        passes=passes,
                        iterations=iterations,
                        random_state=241,
                        chunksize=2000,
                        per_word_topics=False
                    )
                    
                    # Calcul de la cohérence
                    coherence_model_lda = CoherenceModel(model=lda_model, texts=text_data, dictionary=dictionary, coherence='c_v')
                    coherence_score = coherence_model_lda.get_coherence()

                    # Stockage des résultats
                    results.append({
                        'num_topics': num_topics,
                        'alpha': alpha,
                        'eta': eta,
                        'passes': passes,
                        'iterations': iterations,
                        'coherence': coherence_score
                    })

                    print(f"num_topics={num_topics}, alpha={alpha}, eta={eta}, passes={passes}, coherence={coherence_score:.4f}")


num_topics=5, alpha=symmetric, eta=symmetric, passes=10, coherence=0.3249
num_topics=5, alpha=symmetric, eta=symmetric, passes=20, coherence=0.3283
num_topics=5, alpha=asymmetric, eta=symmetric, passes=10, coherence=0.3636
num_topics=5, alpha=asymmetric, eta=symmetric, passes=20, coherence=0.3703
num_topics=10, alpha=symmetric, eta=symmetric, passes=10, coherence=0.3503
num_topics=10, alpha=symmetric, eta=symmetric, passes=20, coherence=0.3481
num_topics=10, alpha=asymmetric, eta=symmetric, passes=10, coherence=0.3932
num_topics=10, alpha=asymmetric, eta=symmetric, passes=20, coherence=0.3836
num_topics=15, alpha=symmetric, eta=symmetric, passes=10, coherence=0.3754
num_topics=15, alpha=symmetric, eta=symmetric, passes=20, coherence=0.3827
num_topics=15, alpha=asymmetric, eta=symmetric, passes=10, coherence=0.3717
num_topics=15, alpha=asymmetric, eta=symmetric, passes=20, coherence=0.3814
num_topics=20, alpha=symmetric, eta=symmetric, passes=10, coherence=0.3941
num_topics=20, alpha=sy

In [ ]:
# Entrainement du meilleur modele
best_model = max(results, key=lambda x: x['coherence'])
# print("Meilleure config :", best_model)

best_lda_model = models.LdaModel(
    corpus=corpus_bow, # Le corpus d'entraînement sous forme de BoW (liste de (token_id, count) par proposition) 
    id2word=dictionary, # Dictionnaire de correspondance token_id -> token
    num_topics=best_model['num_topics'], # Nombre de thèmes à extraire
    alpha=best_model['alpha'], eta=best_model['eta'], # Paramètres de régularisation
    passes=best_model['passes'], # Nombre de passes/passages sur le corpus - plus il est élevé, plus le modèle converge, plus il est précis
    iterations=best_model['iterations'], # Nombre d’itérations par proposition par passe
    random_state=241
)

# Visualisation du meilleur modele avec pyLDAvis
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim_models.prepare(best_lda_model, corpus_bow, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

## Scoring des thèmes

In [15]:
# On attribue le.s thème.s dominant.s à chaque proposition
    # si on a plusieurs thèmes proches, on les attribue tous en dupliquant la proposition

# Seuil de proximité entre les probabilités
epsilon = 0.02

# Probabilités des thèmes pour chaque document
proba_themes = best_lda_model.get_document_topics(corpus_bow, minimum_probability=0)

# Pour chaque ligne, on extrait les thèmes proches du max
rows = []
for i, doc_probs in enumerate(proba_themes):
    probs = np.array([prob for _, prob in doc_probs])
    max_prob = probs.max()
    
    # On identifie tous les thèmes "quasi-dominants"
    close_themes = [topic_id for topic_id, prob in doc_probs if abs(prob - max_prob) <= epsilon]
    
    for topic in close_themes:
        row = data.iloc[i].copy()
        row['theme_dominant'] = topic
        row['theme_proba'] = dict(doc_probs)[topic]
        row['theme_count'] = len(close_themes)
        rows.append(row)

# Créer un nouveau DataFrame avec duplications si besoin
data_themes = pd.DataFrame(rows)
data_themes

,ID,USER_ID,USER_NAME,USER_AGE,USER_TYPE,USER_CP,CONTENT,SLUG,COUNT_AGREE,COUNT_LIKE_IT,...,COUNT_DISAGREE,COUNT_IMPOSSIBLE,COUNT_NO_WAY,COUNT_PLATITUDE_DISAGREE,USER_DEPARTEMENT,USER_REGION,CONTENT_PROCESSED,theme_dominant,theme_proba,theme_count
0,67d0c562-fb00-4228-aa08-e504a8077c38,93ef60de-5827-49f1-bd56-e1f580619540,Dominique,64.0,USER,None,Il faut favoriser le stationnement des véhicul...,il-faut-favoriser-le-stationnement-des-vehicul...,46,8,...,30,4,13,4,No,Autre,favoriser stationnement vehicule loisir site t...,0,0.909804,1
1,9798263d-96a3-4e35-9ba2-f15c6f6402d8,dd920812-f535-438d-a1d2-f7c72a24bc98,bosco,20.0,USER,None,Il faut éviter la course aux équipements et à ...,il-faut-eviter-la-course-aux-equipements-et-a-...,52,9,...,33,4,11,5,No,Autre,eviter course equipement densification offre h...,1,0.892967,1
2,885d953e-a29a-4323-96d8-f149d3e512c7,38cf0349-3b79-4f4a-8c8f-14bfe1d3a854,Virginie,49.0,USER,63000,Il faut remettre en service les petites lignes...,il-faut-remettre-en-service-les-petites-lignes...,591,166,...,90,43,23,13,63,Auvergne-Rhône-Alpes,remettre service petit ligne sncf interieure f...,8,0.894682,1
3,6a660958-f5c7-44ce-83de-d165263c8986,c7846aa6-6c28-4d27-8f75-4eb3f21c2054,Sabrina,NaN,USER,None,Il faut taxer de manière significative tous le...,il-faut-taxer-de-maniere-significative-tous-le...,91,26,...,36,7,19,7,No,Autre,taxer maniere significatif vol court national ...,2,0.688586,1
4,fb259cf7-ec19-459b-a6ad-2254689b14c3,39ae8c6b-e853-48a9-ad03-150ff2009f9e,Bruno,61.0,USER,31240,Il faut un questionnaire obligatoire à chaque ...,il-faut-un-questionnaire-obligatoire-a-chaque-...,70,11,...,77,29,20,11,31,Occitanie,questionnaire obligatoire reservation informe ...,1,0.914403,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,01fc2034-32a7-4bf2-8819-41d4da2b7781,76a6dc53-8cde-4bf1-8f3b-aadfa9eda086,Clémence,36.0,USER,None,"Il faut ""éduquer"" les voyageurs sur les choix ...",il-faut-eduquer-les-voyageurs-sur-les-choix-ex...,35,7,...,11,3,5,4,No,Autre,eduquer voyageur choix existant limiter emprei...,9,0.932040,1
1492,c1331531-03e6-4d16-8466-f9b23196c842,1b66a5a4-34b8-4e4d-b87a-085fcb89aee1,Yann,50.0,USER,75008,Il faut réduire l'utilisation de la voiture in...,il-faut-reduire-l-utilisation-de-la-voiture-in...,67,14,...,42,19,13,8,75,Île-de-France,reduire utilisation voiture individuel encoura...,0,0.909975,1
1493,9eb50171-5678-44db-ad69-2ffab2121553,b2b6e313-a1b3-4ed8-810c-130f0652b8db,Marie,30.0,USER,None,Il faut interdire la baignade sur les plages q...,il-faut-interdire-la-baignade-sur-les-plages-q...,56,11,...,21,8,9,11,No,Autre,interdire baignade plage eau trop pollue usage...,8,0.913714,1
1494,5b423680-f77b-42aa-adfe-c32e6bc1c1d3,92858108-3377-410c-9c2d-2c3318d3e24c,Nicolas,NaN,USER,None,Il faut que les intercommunalités aient un pla...,il-faut-que-les-intercommunalites-aient-un-pla...,39,6,...,10,7,3,4,No,Autre,intercommunalite avoir plan interconnexion dec...,1,0.934043,1


In [16]:
# Définir les poids pour le calcul du score_votes
poids_votes = {
    'COUNT_DISAGREE': -1,
    'COUNT_IMPOSSIBLE': -5,
    'COUNT_NO_WAY': -2.5,
    'COUNT_PLATITUDE_DISAGREE': -1.5,
    'COUNT_AGREE': 1,
    'COUNT_LIKE_IT': 5,
    'COUNT_PLATITUDE_AGREE': 2.5,
    'COUNT_DOABLE': 1.5,
    'COUNT_NEUTRAL': 0,
    'COUNT_NO_OPINION': 1,
    'COUNT_DN_UNDERSTAND': -1,
    'COUNT_DN_CARE': 0
}

# Regrouper par thème
grouped = data_themes.groupby('theme_dominant')

# Calcul du score_votes
score_votes = grouped[list(poids_votes.keys())].sum()
score_votes = score_votes.mul(poids_votes).sum(axis=1)

# Calcul du score_controverse
somme_agree = grouped['COUNT_AGREE'].sum()
somme_disagree = grouped['COUNT_DISAGREE'].sum()
score_controverse = somme_disagree / (somme_agree + somme_disagree)

# Création de la nouvelle DataFrame
data_theme_scores = pd.DataFrame({
    'numero_theme': score_votes.index,
    'score_votes': score_votes.values,
    'score_controverse': score_controverse.values
}).reset_index(drop=True)


In [ ]:
# Définir les poids pour le calcul du score_votes
poids_votes = {
    'COUNT_DISAGREE': -1,
    'COUNT_IMPOSSIBLE': -5,
    'COUNT_NO_WAY': -2.5,
    'COUNT_PLATITUDE_DISAGREE': -1.5,
    'COUNT_AGREE': 1,
    'COUNT_LIKE_IT': 5,
    'COUNT_PLATITUDE_AGREE': 2.5,
    'COUNT_DOABLE': 1.5,
    'COUNT_NEUTRAL': 0,
    'COUNT_NO_OPINION': 1,
    'COUNT_DN_UNDERSTAND': -1,
    'COUNT_DN_CARE': 0
}

# Regrouper par thème
grouped = data_themes.groupby('theme_dominant')

# Calcul des sommes par thème pour chaque type de vote
votes_sum = grouped[list(poids_votes.keys())].sum()

# Score pondéré
score_pondere = votes_sum.mul(poids_votes).sum(axis=1)

# Total des sous-votes (nombre total de votes pour chaque thème)
total_votes = votes_sum.sum(axis=1)

# Score normalisé
score_votes_normalise = score_pondere / total_votes

# Calcul du score_controverse
somme_agree = grouped['COUNT_AGREE'].sum()
somme_disagree = grouped['COUNT_DISAGREE'].sum()
score_controverse = somme_disagree / (somme_agree + somme_disagree)

# Création de la nouvelle DataFrame
data_theme_scores = pd.DataFrame({
    'numero_theme': score_votes_normalise.index,
    'score_votes': score_votes_normalise.values,
    'score_controverse': score_controverse.values
}).reset_index(drop=True)

In [18]:
data_theme_scores

,numero_theme,score_votes,score_controverse
0,0,0.699003,0.181235
1,1,0.841461,0.134624
2,2,0.746087,0.174137
3,3,0.918967,0.129383
4,4,0.649731,0.194776
5,5,0.584697,0.209420
6,6,0.861405,0.137639
7,7,0.848162,0.139516
8,8,0.698962,0.186942
9,9,0.439612,0.263973


# Extraction des actions principales

In [19]:
data_theme_final = data_themes.loc[data_themes['theme_dominant'].isin([3, 6, 7, 1]), ['ID', 'CONTENT', 'CONTENT_PROCESSED', 'theme_dominant']]
data_theme_final # 500 propositions

,ID,CONTENT,CONTENT_PROCESSED,theme_dominant
1,9798263d-96a3-4e35-9ba2-f15c6f6402d8,Il faut éviter la course aux équipements et à ...,eviter course equipement densification offre h...,1
4,fb259cf7-ec19-459b-a6ad-2254689b14c3,Il faut un questionnaire obligatoire à chaque ...,questionnaire obligatoire reservation informe ...,1
7,a8608bf3-905a-40c8-a2a8-ddb958ecae29,Il faut favoriser les projets d'écotourisme pa...,favoriser projet ecotourisme participatif but ...,3
13,e5f9e4e6-316e-4be2-8e8a-19a0fcfac158,Il faut revenir aux fondamentaux des vacances ...,revenir fondamentau vacance contemplation isol...,7
15,74d5a1ff-6af3-4010-9df7-5f542c47dd5a,Il faut permettre en France le stationnement e...,permettre france stationnement aire repos fonc...,1
...,...,...,...,...
1481,2c99d04f-59b5-4aba-aa18-0979311e886e,Il faut prévoir davantage d'emplacements pour ...,prevoir davantage emplacement velo train perme...,1
1485,c5808faf-a696-494b-bb0a-2c54b2f87b5c,"Il faut promouvoir les achats locaux (marchés,...",promouvoir achat local marche producteur taxer...,3
1488,298ed4e5-b87d-4e2e-8064-f8f987ed9dcf,Il faut que les produits soit le plus local po...,produit local,3
1494,5b423680-f77b-42aa-adfe-c32e6bc1c1d3,Il faut que les intercommunalités aient un pla...,intercommunalite avoir plan interconnexion dec...,1


## Traduction des thèmes

In [ ]:
def summariser_par_theme(data, k=10):
    """
    Pour chaque thème dans data['theme_dominant'], fait :
      - TF-IDF vectorisation des textes
      - grid search de K dans [min_k .. min(max_k, n_textes-1)] par silhouette
      - clustering KMeans
      - sélection de la phrase la plus proche du centroïde pour chaque cluster
    Retourne un dict {theme: [résumé_cluster_1, ..., résumé_cluster_K]}
    """
    summaries = {}
    vectorizer = TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1,2))
    
    for theme, group in data.groupby('theme_dominant'):
        textes = group['CONTENT_PROCESSED'].tolist()
        n = len(textes)
        if n < k:
            summaries[theme] = ["(pas assez de propositions pour clustériser)"]
            continue
        
        # TF-IDF
        X = vectorizer.fit_transform(textes)
       
        # Refit KMeans avec le meilleur K
        km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X)
        labels = km.labels_
        centroids = km.cluster_centers_
        
        # Pour chaque cluster, trouver l'indice de la phrase la plus proche du centroïde
        distances = euclidean_distances(X, centroids)
        # distances[i,j] = distance du texte i au centro j
        reps = []
        for cluster_id in range(k):
            # sous-matrice pour cluster j
            idx_cluster = (labels == cluster_id)
            # trouver i minimisant distance à centroids[cluster_id]
            closest = distances[idx_cluster, cluster_id].argmin()
            # récupérer le texte original
            i_global = [i for i, lbl in enumerate(labels) if lbl == cluster_id][closest]
            reps.append(textes[i_global])
        
        summaries[theme] = reps
    
    return summaries

In [21]:
n_clusters_by_theme = {
    theme: max(2, int(ceil((data_theme_final['theme_dominant'] == theme).sum() / 37))+1)
    for theme in data_theme_final['theme_dominant'].unique()
}

# Appel de la fonction pour chaque thème avec le bon nombre de clusters
summaries = {}
for theme, k in n_clusters_by_theme.items():
    group = data_theme_final[data_theme_final['theme_dominant'] == theme]
    summaries[theme] = summariser_par_theme(group, k=k)[theme]

# Affichage des résumés par thème
for theme, reps in summaries.items():
    print(f"\n=== Thème {theme} (K={len(reps)}) (NB={(data_theme_final['theme_dominant'] == theme).sum()}) ===")
    for phrase in reps:
        print(f" • {phrase}")



=== Thème 1 (K=9) (NB=271) ===
 • arreter emballage plastique preferer conditionnement verre consigne
 • apprendre responsable quotidien individu responsable quotidien responsable
 • pouvoir contribuer preservation biodiversite vacance
 • mettre disposition poubelle lieux
 • donner moyen ambition faire france premier destination mondial velo
 • developper aire camping car rehabiliter camping municipal valoriser accueil ferme producteur
 • proposition voyage fass objet mesure empreindre environnemental
 • soumettre aide collectivite eco-conditionnalite label gestion durable
 • faire geste simple utiliser poubelle mise disposition apprendre enfant faire

=== Thème 3 (K=5) (NB=133) ===
 • cesser construire faire naviguer paquebot croisiere pouvoir accueillir 10000 personne
 • habitant devenir acteur territoire
 • proposer voyage velo location velo depart point donne point
 • consommer local privilegier acteur produit local tpe local acheter local
 • mettre poubelle eviter abandon dechet


## Extraction des actions par thème

> Thème 3 (K=5) (NB=133) : **0.92 & 0.13**

- Limiter les paquebots de croisière géants.

- Impliquer les habitants dans le développement de leur territoire.

- Faciliter les voyages à vélo, avec location et itinéraires flexibles.

- Favoriser la consommation locale.

- Installer plus de poubelles pour lutter contre les déchets abandonnés.

> Thème 6 (K=3) (NB=59) : **0.86 & 0.14**

- Limiter l'accès aux sites naturels pour les préserver.

- Faciliter l’accès aux centres-villes tout en respectant l’environnement.

- L’écologie doit être sincère et non une opération de communication.

> Thème 7 (K=2) (NB=37) : **0.85 & 0.14**

- Obliger les grands sites à installer des fontaines et interdire la vente de bouteilles en plastique.

- Faciliter les démarches administratives en zone rurale de façon responsable et équitable.


> Thème 1 (K=9) (NB=271) : **0.84 & 0.13**

- Arrêter les emballages plastiques, privilégier le verre consigné.

- Apprendre les gestes responsables au quotidien.

- Agir pour la biodiversité même pendant les vacances.

- Installer plus de poubelles dans les lieux publics.

- Faire de la France une grande destination du tourisme à vélo.

- Valoriser le tourisme rural : aires de camping-cars, campings municipaux, fermes accueillantes.

- Mesurer l’impact environnemental des voyages.

- Conditionner les aides aux collectivités à des pratiques durables.

- Apprendre les bons gestes écologiques dès l’enfance.

## Sélection des actions principales

- Limiter les paquebots de croisière géants.
- Valoriser le tourisme rural : aires de camping-cars, campings municipaux, fermes accueillantes.
---

- Faciliter les voyages à vélo, avec location et itinéraires flexibles.
- Faire de la France une grande destination du tourisme à vélo.
---

- Installer plus de poubelles pour lutter contre les déchets abandonnés.
- Obliger les grands sites à installer des fontaines et interdire la vente de bouteilles en plastique.
- Arrêter les emballages plastiques, privilégier le verre consigné.
- Installer plus de poubelles dans les lieux publics.
---

- Apprendre les gestes responsables au quotidien.
- Apprendre les bons gestes écologiques dès l’enfance.